In [1]:
#Importamos librerias
from pathlib import Path
from venv import logger
import geopandas as gpd
import pandas as pd
import osmnx as ox
from sklearn.cluster import DBSCAN
import yaml
import numpy as np
from shapely.geometry import Polygon
from concave_hull import concave_hull
from datetime import datetime
import os





In [63]:
# Definimos los parámetros de DBSCAN para cada tipo de categoría
# Cada clave del diccionario 'parametros' corresponde a un 'param_type' en 'osm_macro_categories'

parametros = {0:{'eps': 200, 
                         'min_samples': 30} ,
                     1: {'eps': 400,
                         'min_samples': 20},
                     2: {'eps': 400, 
                         'min_samples': 30}
                               }

In [78]:
osm_macro_categories[['macro_category', 'param_type']].drop_duplicates().reset_index(drop=True).sort_values('param_type')

,macro_category,param_type
10,Otros,0
1,education_and_culture,1
5,health_and_beauty,1
6,finance_and_corporate,1
4,sports_and_outdoor,1
3,daily_supply_services,2
0,gastronomy,2
2,fashion_and_clothes,2
7,automotive_and_transport,2
8,tourism_and_entertainment,2


In [5]:
PROCESSED_DATA_DIR = Path(os.getcwd()).parent / "data/processed"

# cargamos los datos procesados

gdf_osm_pois = gpd.read_parquet(PROCESSED_DATA_DIR / "osm_pois.parquet")        

osm_macro_categories = pd.read_csv(PROCESSED_DATA_DIR / "osm_pois_categorized.csv" )

# Unir gdf_osm_pois con osm_macro_categories para obtener 'macro_category' y 'param_type' para cada POI
gdf_osm_pois = gdf_osm_pois.merge(osm_macro_categories[['sub_tag', 'macro_category','param_type']], on='sub_tag', how='left')
                            

In [6]:
# convertimos a una proyeccion metricas
gdf_osm_pois['geometry_2'] = gdf_osm_pois.to_crs('EPSG:3857').geometry.centroid
# Extraemos coordenadas x e y de geometry (lo requiere la api de DBSCAN)
gdf_osm_pois['x'] = gdf_osm_pois.geometry_2.x
gdf_osm_pois['y'] = gdf_osm_pois.geometry_2.y

# ordenamos por de norte a sur y de oeste a este
gdf_osm_pois.sort_values(['x', 'y'], inplace=True)

# preparamos la matriz de coordenadas
X = gdf_osm_pois.loc[:,['y','x']].values

In [7]:
gdf_osm_pois

,osmid,tag,tipo_osm,nombre_osm,sub_tag,timestamp,geometry,macro_category,param_type,geometry_2,x,y
13899,None,leisure,None,Paseo de los Granaderos,park,2025-12-03 17:42:49,POINT (-58.53404 -34.61643),sports_and_outdoor,1,POINT (-6515979.96 -4111876.888),-6.515980e+06,-4.111877e+06
10525,None,amenity,None,Credicoop,bank,2025-12-03 17:42:44,POINT (-58.53174 -34.61843),finance_and_corporate,1,POINT (-6515723.269 -4112146.815),-6.515723e+06,-4.112147e+06
2744,None,amenity,None,Su Remis,taxi,2025-12-03 17:42:44,POINT (-58.53174 -34.61728),automotive_and_transport,2,POINT (-6515723.058 -4111991.798),-6.515723e+06,-4.111992e+06
18351,None,shop,None,Auto Special,car,2025-12-03 17:44:12,POINT (-58.53167 -34.6182),automotive_and_transport,2,POINT (-6515715.466 -4112116.041),-6.515715e+06,-4.112116e+06
17694,None,shop,None,Snack 24HS,convenience,2025-12-03 17:44:12,POINT (-58.53163 -34.61728),daily_supply_services,2,POINT (-6515711.436 -4111992.231),-6.515711e+06,-4.111992e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
12140,None,amenity,None,Cuartel Isla Demarchi,fire_station,2025-12-03 17:42:44,POINT (-58.35053 -34.62775),Otros,0,POINT (-6495551.761 -4113408.048),-6.495552e+06,-4.113408e+06
11271,None,amenity,None,"Escuela Nacional Fluvial ""Comodoro Antonio Som...",college,2025-12-03 17:42:44,POINT (-58.34995 -34.62921),education_and_culture,1,POINT (-6495486.207 -4113605.575),-6.495486e+06,-4.113606e+06
32973,None,industrial,None,Terminal Portuaria Exolgan,port,2025-12-03 18:11:03,POINT (-58.34746 -34.641),Otros,0,POINT (-6495209.227 -4115201.403),-6.495209e+06,-4.115201e+06
32977,None,industrial,None,Complejo Industrial y Naval Argentino,shipyard,2025-12-03 18:11:03,POINT (-58.34686 -34.62638),Otros,0,POINT (-6495142.78 -4113223.367),-6.495143e+06,-4.113223e+06


In [8]:
# entrenamos el modelo
dbscan_lineas = DBSCAN(
    eps=200,
    min_samples=30,
    metric='manhattan'
).fit(X)
  

In [9]:
# Asignamos las etiquetas de cluster al geodataframe de pois según el resultado de DBSCAN - usamos indexing por loc para evitar SettingWithCopyWarning
gdf_osm_pois.loc[:, 'cluster'] = dbscan_lineas.labels_
# Reordenamos las etiquetas de cluster para que vayan de 0 a n_clusters-1, dejando -1 para ruido
etiquetas_clusters = gdf_osm_pois.cluster.value_counts().index[gdf_osm_pois.cluster.value_counts().index > -1]
# Creamos un diccionario que mapea las etiquetas originales a las nuevas etiquetas ordenadas por tamaño de cluster
etiquetas_por_tamanio = {k: v for k, v in zip(etiquetas_clusters, range(len(etiquetas_clusters)))}
# Reemplazamos las etiquetas en el geodataframe usando el diccionario de mapeo
gdf_osm_pois.cluster = gdf_osm_pois.cluster.replace(etiquetas_por_tamanio)


In [10]:
# ahora vamos a crear los poligonos de los clusters
# cada coordenada x,y de un poi se convierte en un punto del futuro poligono del cluster
gdf_osm_pois['x_y_concat'] = list(zip(gdf_osm_pois['x'], gdf_osm_pois['y'])) #  quizás esta linea es redundante
# ahora agrupamos por cluster y creamos una lista de puntos x,y para cada cluster
bordes_clusters = gdf_osm_pois.groupby(['cluster'])['x_y_concat'].agg(list).reset_index()
# eliminamos el cluster -1 (ruido)
bordes_clusters = bordes_clusters.loc[bordes_clusters.cluster != -1].reset_index(drop=True)


In [11]:

# ahora creamos los poligonos con concave hull
for index, row in bordes_clusters.iterrows():
    # Si el número de puntos en el cluster es mayor a 4, calculamos el polígono concavo
    # Si el número de puntos es menor o igual a 4, asignamos un polígono vacío
    # Esto es porque un polígono concavo requiere al menos 5 puntos para ser definido
    # Si hay menos de 5 puntos, no podemos calcular un polígono concavo
    # y asignamos un polígono vacío para evitar errores
    if len(bordes_clusters.x_y_concat[index]) > 4: 
        # Convertimos las coordenadas concatenadas en un formato adecuado para 'concave_hull'
        # 'concave_hull' espera una lista de puntos, por lo que convertimos
        # las coordenadas concatenadas en una lista de tuplas
        puntos = bordes_clusters.x_y_concat[index]
        # Calculamos el polígono concavo utilizando la el indice de concavidad 2
        # ver en libreria de concave_hull 
        bordes_clusters.at[index, 'geometry'] = Polygon(concave_hull(puntos, concavity = 2))
    else:
        bordes_clusters.at[index, 'geometry'] = Polygon()

In [12]:
# Convertimos 'bordes_clusters' a un GeoDataFrame que contiene los bordes de cada cluster
# Eliminamos la columna 'x_y_concat' que ya no es necesaria
bordes_clusters.drop(columns='x_y_concat',inplace=True)
bordes_clusters = gpd.GeoDataFrame(bordes_clusters, geometry='geometry')
# Asignamos el sistema de coordenadas a 'bordes_clusters'
bordes_clusters.crs = 'EPSG:3857'
bordes_clusters = bordes_clusters.to_crs("EPSG:4326")

In [13]:
import folium
from folium.plugins import MarkerCluster

center = [-34.61, -58.38]  # CABA approx
m = folium.Map(location=center, zoom_start=12, tiles="cartodbpositron")

# capas de polígonos (ej. barrios)
folium.GeoJson(bordes_clusters.to_json(),name="clusters", style_function=lambda feat: {"color":"#444444","weight":1,"fillOpacity":0.1}
                  ).add_to(m)

In [15]:
osm_macro_categories

,sub_tag,count,macro_category,param_type
0,restaurant,2376,gastronomy,2
1,school,1730,education_and_culture,1
2,cafe,1486,gastronomy,2
3,clothes,1390,fashion_and_clothes,2
4,fast_food,1087,gastronomy,2
...,...,...,...,...
431,men;discount,1,Otros,0
432,sports;underwear,1,fashion_and_clothes,2
433,women;hats;underwear,1,fashion_and_clothes,2
434,women;men;underwear;wedding;children;maternity...,1,fashion_and_clothes,2


In [66]:
# creamos un mapeo de macro_category a un índice numérico para iterar
macro_category_to_index = {category: idx for idx, category in enumerate(gdf_osm_pois['macro_category'].unique())}
gdf_osm_pois['macro_category_index'] = gdf_osm_pois['macro_category'].map(macro_category_to_index)

# Creamos un DataFrame vacío para almacenar los clusters
clusters_especiales = pd.DataFrame()

# Iteramos sobre cada corona y aplicamos DBSCAN
# para identificar los clusters de POIs
for category in sorted(gdf_osm_pois['macro_category_index'].unique()):
    pois_category = gdf_osm_pois.loc[gdf_osm_pois['macro_category_index'] == category].copy()
    pois_category.sort_values(['x', 'y'], inplace=True)

    # obtenemos los parametros de DBSCAN para la categoria actual
    param_type = pois_category['param_type'].iloc[0]
    eps = parametros[param_type]['eps']
    min_samples = parametros[param_type]['min_samples']

    # preparamos la matriz de coordenadas
    X = pois_category.loc[:,['y','x']].values
    # entrenamos el modelo
    dbscan = DBSCAN(
        eps=eps,
        min_samples=min_samples,
        metric='manhattan'
    ).fit(X)

    # Asignamos las etiquetas de cluster al geodataframe de pois según el resultado de DBSCAN - usamos indexing por loc para evitar SettingWithCopyWarning
    pois_category.loc[:, 'cluster_special'] = dbscan.labels_
    # Reordenamos las etiquetas de cluster para que vayan de 0 a n_clusters-1, dejando -1 para ruido
    etiquetas_clusters = pois_category.cluster_special.value_counts().index[pois_category.cluster_special.value_counts().index > -1]
    # Creamos un diccionario que mapea las etiquetas originales a las nuevas etiquetas ordenadas por tamaño de cluster
    etiquetas_por_tamanio = {k: v for k, v in zip(etiquetas_clusters, range(len(etiquetas_clusters)))}
    # Reemplazamos las etiquetas en el geodataframe usando el diccionario de mapeo
    pois_category.cluster_special = pois_category.cluster_special.replace(etiquetas_por_tamanio)            
    # Concatenamos los resultados con el DataFrame de clusters_especiales
    clusters_especiales = pd.concat([clusters_especiales, pois_category], ignore_index=True)

clusters_especiales


,osmid,tag,tipo_osm,nombre_osm,sub_tag,timestamp,geometry,macro_category,param_type,geometry_2,x,y,cluster,x_y_concat,macro_category_index,cluster_special
0,None,leisure,None,Paseo de los Granaderos,park,2025-12-03 17:42:49,POINT (-58.53404 -34.61643),sports_and_outdoor,1,POINT (-6515979.96 -4111876.888),-6.515980e+06,-4.111877e+06,-1,"(-6515979.960071956, -4111876.887666181)",0,-1
1,None,leisure,None,Plaza Ramón Lista,park,2025-12-03 17:42:49,POINT (-58.53019 -34.65051),sports_and_outdoor,1,POINT (-6515551.456 -4116487.364),-6.515551e+06,-4.116487e+06,-1,"(-6515551.455617666, -4116487.3640303584)",0,-1
2,None,leisure,None,"Plaza Héctor ""Turco"" Carnip",park,2025-12-03 17:42:49,POINT (-58.53014 -34.65199),sports_and_outdoor,1,POINT (-6515545.556 -4116687.62),-6.515546e+06,-4.116688e+06,-1,"(-6515545.556187284, -4116687.619827189)",0,-1
3,None,leisure,None,Espacio Brian Lucero,park,2025-12-03 17:42:49,POINT (-58.53009 -34.65201),sports_and_outdoor,1,POINT (-6515540.36 -4116690.163),-6.515540e+06,-4.116690e+06,-1,"(-6515540.360349979, -4116690.1629284765)",0,-1
4,None,leisure,None,Amigos,sports_centre,2025-12-03 17:42:49,POINT (-58.52989 -34.61886),sports_and_outdoor,1,POINT (-6515517.819 -4112205.604),-6.515518e+06,-4.112206e+06,6,"(-6515517.819389283, -4112205.6037565456)",0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33435,None,amenity,None,Capilla María Madre La Esperanza,place_of_worship,2025-12-03 17:42:44,POINT (-58.35446 -34.63338),education_and_culture,1,POINT (-6495989.073 -4114169.636),-6.495989e+06,-4.114170e+06,-1,"(-6495989.073278949, -4114169.635666075)",10,-1
33436,None,tourism,None,Madre Marinera,artwork,2025-12-03 17:44:07,POINT (-58.3533 -34.63436),education_and_culture,1,POINT (-6495859.932 -4114303.089),-6.495860e+06,-4.114303e+06,-1,"(-6495859.931537807, -4114303.089080169)",10,-1
33437,None,amenity,None,Nuestra Señora de los Milagros de Caacupé,place_of_worship,2025-12-03 17:42:44,POINT (-58.35308 -34.61868),education_and_culture,1,POINT (-6495834.647 -4112181.52),-6.495835e+06,-4.112182e+06,-1,"(-6495834.647113812, -4112181.5199902244)",10,-1
33438,None,amenity,None,Escuela De Educación Primaria Nº6 Cnel. De Mar...,school,2025-12-03 17:42:44,POINT (-58.35182 -34.64194),education_and_culture,1,POINT (-6495694.533 -4115327.665),-6.495695e+06,-4.115328e+06,-1,"(-6495694.533038384, -4115327.6648264024)",10,-1


In [67]:
# ahora vamos a crear los poligonos de los clusters
# cada coordenada x,y de un poi se convierte en un punto del futuro poligono del cluster
clusters_especiales['x_y_concat'] = list(zip(clusters_especiales['x'], clusters_especiales['y'])) #  quizás esta linea es redundante
# ahora agrupamos por cluster y creamos una lista de puntos x,y para cada cluster
bordes_clusters_especiales = clusters_especiales.groupby(['macro_category_index','cluster_special'])['x_y_concat'].agg(list).reset_index()
# eliminamos el cluster -1 (ruido)
bordes_clusters_especiales = bordes_clusters_especiales.loc[bordes_clusters_especiales.cluster_special != -1].reset_index(drop=True)



In [68]:
# ahora creamos los poligonos con concave hull
for index, row in bordes_clusters_especiales.iterrows():
    # Si el número de puntos en el cluster es mayor a 4, calculamos el polígono concavo
    # Si el número de puntos es menor o igual a 4, asignamos un polígono vacío
    # Esto es porque un polígono concavo requiere al menos 5 puntos para ser definido
    # Si hay menos de 5 puntos, no podemos calcular un polígono concavo
    # y asignamos un polígono vacío para evitar errores
    if len(bordes_clusters_especiales.x_y_concat[index]) > 4: 
        # Convertimos las coordenadas concatenadas en un formato adecuado para 'concave_hull'
        # 'concave_hull' espera una lista de puntos, por lo que convertimos
        # las coordenadas concatenadas en una lista de tuplas
        puntos = bordes_clusters_especiales.x_y_concat[index]
        # Calculamos el polígono concavo utilizando la el indice de concavidad 2
        # ver en libreria de concave_hull 
        bordes_clusters_especiales.at[index, 'geometry'] = Polygon(concave_hull(puntos, concavity = 2))
    else:
        bordes_clusters_especiales.at[index, 'geometry'] = Polygon()

In [69]:
# Convertimos 'bordes_clusters' a un GeoDataFrame que contiene los bordes de cada cluster
# Eliminamos la columna 'x_y_concat' que ya no es necesaria
bordes_clusters_especiales.drop(columns='x_y_concat',inplace=True)
bordes_clusters_especiales = gpd.GeoDataFrame(bordes_clusters_especiales, geometry='geometry')
# Asignamos el sistema de coordenadas a 'bordes_clusters'
bordes_clusters_especiales.crs = 'EPSG:3857'
bordes_clusters_especiales = bordes_clusters_especiales.to_crs("EPSG:4326")

In [87]:
bordes_clusters_especiales

,macro_category_index,cluster_special,geometry,macro_category
0,1,0,"POLYGON ((-58.39059 -34.60079, -58.38967 -34.6...",finance_and_corporate
1,1,1,"POLYGON ((-58.52825 -34.63944, -58.5282 -34.63...",finance_and_corporate
2,1,2,"POLYGON ((-58.45901 -34.56494, -58.45834 -34.5...",finance_and_corporate
3,1,3,"POLYGON ((-58.48725 -34.57501, -58.48782 -34.5...",finance_and_corporate
4,1,4,"POLYGON ((-58.47284 -34.54542, -58.47273 -34.5...",finance_and_corporate
...,...,...,...,...
76,10,7,"POLYGON ((-58.46386 -34.6288, -58.46382 -34.62...",education_and_culture
77,10,8,"POLYGON ((-58.42022 -34.62456, -58.42004 -34.6...",education_and_culture
78,10,9,"POLYGON ((-58.37862 -34.61918, -58.38012 -34.6...",education_and_culture
79,10,10,"POLYGON ((-58.42601 -34.61501, -58.42564 -34.6...",education_and_culture


In [70]:
macro_category_to_index



{'sports_and_outdoor': 0,
 'finance_and_corporate': 1,
 'automotive_and_transport': 2,
 'daily_supply_services': 3,
 'Otros': 4,
 'health_and_beauty': 5,
 'fashion_and_clothes': 6,
 'gastronomy': 7,
 'tourism_and_entertainment': 8,
 'home_and_construction': 9,
 'education_and_culture': 10}

In [75]:
reversed_dict = {value: key for key, value in macro_category_to_index.items()}
reversed_dict

bordes_clusters_especiales['macro_category'] = bordes_clusters_especiales['macro_category_index'].map(reversed_dict)


In [80]:
import folium

center = [-34.61, -58.38]
m = folium.Map(location=center, zoom_start=12, tiles="cartodbpositron")


In [81]:

for cat in bordes_clusters_especiales['macro_category'].unique():
    fg = folium.FeatureGroup(name=cat)
    gdf_cat = bordes_clusters_especiales[bordes_clusters_especiales['macro_category'] == cat]
    folium.GeoJson(
        gdf_cat.to_json(),
        style_function=lambda feat: {"color": "#444444", "weight": 1, "fillOpacity": 0.1},
        popup=folium.GeoJsonPopup(fields=["macro_category"], labels=True)
    ).add_to(fg)
    fg.add_to(m)

folium.LayerControl().add_to(m)

In [86]:

OUTPUT_DATA_DIR = Path(os.getcwd()).parent / "data/outputs/"


m.save(OUTPUT_DATA_DIR / 'mapa.html')